# LDA Tryout

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#LDA-Tryout" data-toc-modified-id="LDA-Tryout-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>LDA Tryout</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Import-DataFrame" data-toc-modified-id="Import-DataFrame-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Import DataFrame</a></span></li><li><span><a href="#Step-1:-CountVectorize-Titles" data-toc-modified-id="Step-1:-CountVectorize-Titles-1.0.2"><span class="toc-item-num">1.0.2&nbsp;&nbsp;</span>Step 1: CountVectorize Titles</a></span><ul class="toc-item"><li><span><a href="#general-content" data-toc-modified-id="general-content-1.0.2.1"><span class="toc-item-num">1.0.2.1&nbsp;&nbsp;</span>general content</a></span></li><li><span><a href="#flood-content-with-NB-classifier" data-toc-modified-id="flood-content-with-NB-classifier-1.0.2.2"><span class="toc-item-num">1.0.2.2&nbsp;&nbsp;</span>flood content with NB classifier</a></span></li></ul></li><li><span><a href="#Building-Corpus" data-toc-modified-id="Building-Corpus-1.0.3"><span class="toc-item-num">1.0.3&nbsp;&nbsp;</span>Building Corpus</a></span></li><li><span><a href="#Running-LDA-with-Num_topics-=-2" data-toc-modified-id="Running-LDA-with-Num_topics-=-2-1.0.4"><span class="toc-item-num">1.0.4&nbsp;&nbsp;</span>Running LDA with Num_topics = 2</a></span></li></ul></li></ul></li></ul></div>

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS
from sklearn.decomposition import LatentDirichletAllocation

from gensim import corpora, models
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

np.random.seed(42)

### Import DataFrame 

In [5]:
df = pd.read_csv('../data/df_balanced_consolidated.csv')

In [25]:
df2 = pd.read_csv('../data/flood_prediction.csv')

In [8]:
df.head()

,author,content,description,publishedAt,source,source_id,source_name,title,types,url,urlToImage,yes_disaster
0,https://www.facebook.com/bbcnews,Image copyrightEPAImage caption\r\n Tens of th...,Storm Pabuk has made landfall in southern Thai...,2019-01-04T07:28:56Z,"{'id': 'bbc-news', 'name': 'BBC News'}",bbc-news,BBC News,Thailand Pabuk: Tourist spots hit by worst sto...,landfall,https://www.bbc.co.uk/news/world-asia-46756239,https://ichef.bbci.co.uk/news/1024/branded_new...,1
1,NEETI UPADHYE,1 U.S. Paul Manaforts Trail of Scandals,Hurricane Michael made landfall in Florida at ...,2018-10-11T16:18:46Z,"{'id': 'the-new-york-times', 'name': 'The New ...",the-new-york-times,The New York Times,‘Trees Flying By’: The Aftermath of Hurricane ...,landfall,https://www.nytimes.com/video/us/politics/1000...,https://static01.nyt.com/images/2018/10/12/us/...,1
2,Dell Cameron,The Federal Communications Commission on Frida...,The Federal Communications Commission on Frida...,2018-10-12T22:25:00Z,"{'id': None, 'name': 'Gizmodo.com'}",NaN,Gizmodo.com,"Widespread Cable, Internet, and Radio Outages ...",landfall,https://gizmodo.com/widespread-cable-internet-...,https://i.kinja-img.com/gawker-media/image/upl...,1
3,Reuters Editorial,(Reuters) - Tropical storm Gordon was making l...,Tropical storm Gordon was making landfall just...,2018-09-05T03:12:45Z,"{'id': 'reuters', 'name': 'Reuters'}",reuters,Reuters,Storm Gordon making landfall west of Alabama-M...,landfall,https://www.reuters.com/article/us-storm-gordo...,https://s4.reutersmedia.net/resources_v2/image...,1
4,Ed Mazza,Stephen Colbert said former New York City Mayo...,"Today's ""Stormy Watch"" finds Hurricane Rudy ""m...",2018-05-04T02:14:53Z,"{'id': 'the-huffington-post', 'name': 'The Huf...",the-huffington-post,The Huffington Post,Colbert Mocks Giuliani’s Bonkers Fox News Inte...,landfall,https://www.huffingtonpost.com/entry/stephen-c...,https://img.huffingtonpost.com/asset/5aebbeac1...,1


In [26]:
df2.head()

,Unnamed: 0,content,Unnamed: 2,Unnamed: 3,prediction_lr,prediction_nb,yes_disaster,title
0,0.0,this story originally appeared on grist and is...,NaN,NaN,0.0,1.0,1.0,people hospitalized after missouri tourist boa...
1,1.0,flash flood can strike with almost no warning ...,NaN,NaN,1.0,1.0,0.0,passenger stranded after iceland s wow air col...
2,2.0,the flood underinsurance problem is arguably t...,NaN,NaN,0.0,0.0,0.0,mgm resort sue victim of la vega shooting s...
3,3.0,this story originally appeared on citylab and ...,NaN,NaN,0.0,1.0,0.0,santa fe high school multiple fatality repor...
4,4.0,scandal read the facebook post of mexican come...,NaN,NaN,0.0,0.0,0.0,ied blast in kashmir km from pulwama terror a...


In [29]:
flood_df = df2[df2['prediction_nb'] == 1][['content', 'prediction_nb']]

In [31]:
flood_df.head()

,content,prediction_nb
0,this story originally appeared on grist and is...,1.0
1,flash flood can strike with almost no warning ...,1.0
3,this story originally appeared on citylab and ...,1.0
7,photo kevin davis left and mayor catherine pug...,1.0
10,image copyright epa image caption local have b...,1.0


In [9]:
title_df = df[['content']]

In [10]:
title_df.head()

,content
0,Image copyrightEPAImage caption\r\n Tens of th...
1,1 U.S. Paul Manaforts Trail of Scandals
2,The Federal Communications Commission on Frida...
3,(Reuters) - Tropical storm Gordon was making l...
4,Stephen Colbert said former New York City Mayo...


### Step 1: CountVectorize Titles 

#### general content 

In [33]:
cv = CountVectorizer(ngram_range = (1, 1), 
                     stop_words = 'english', 
                     min_df = 2)
cv_content = cv.fit_transform(title_df['content'])

In [34]:
cv_content_df = pd.DataFrame(cv_content.toarray(), columns = cv.get_feature_names())

In [35]:
cv_content_df.head()

,00,000,0000,0009,000ft,000km,000m,000mah,000th,000x,...,âł1,âł3,édouard,éireann,élysée,élysées,île,öræfajökull,ąť,ʻōʻō
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
texts = [cv_content_df.columns[cv_content_df.loc[index,:].nonzero()] for index in cv_content_df.index]

/Users/evelyn/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  """Entry point for launching an IPython kernel.


In [18]:
texts[:5]

[Index(['05', '12', '30', '45', 'affect', 'andaman', 'area', 'asked', 'begun',
        'beware', 'caption', 'cause', 'common', 'conditions',
        'copyrightepaimage', 'country', 'crosses', 'days', 'decades',
        'department', 'downpours', 'evacuate', 'expected', 'flash', 'floods',
        'forest', 'gmt', 'gulf', 'heavy', 'hit', 'hitting', 'holiday', 'image',
        'islands', 'koh', 'landfall', 'landslides', 'left', 'likely', 'local',
        'meteorological', 'nakhon', 'officials', 'pabuk', 'peak', 'pelting',
        'people', 'phangan', 'popular', 'possible', 'prepared', 'province',
        'rain', 'rainfalls', 'region', 'rough', 'runoffs', 'said', 'samui',
        'sea', 'seas', 'seaside', 'season', 'severe', 'si', 'south', 'southern',
        'spots', 'storm', 'storms', 'strike', 'tao', 'tens', 'thailand',
        'thammarat', 'thousands', 'time', 'torrential', 'tourist', 'tourists',
        'towns', 'tropical', 'villages', 'warned', 'weaken', 'widespread',
        'wind',

In [19]:
dictionary = corpora.Dictionary(texts)

#### flood content with NB classifier 

In [36]:
cv = CountVectorizer(ngram_range = (1, 1), 
                     stop_words = 'english', 
                     min_df = 2)
cv_flood = cv.fit_transform(flood_df['content'])

In [37]:
cv_flood_df = pd.DataFrame(cv_flood.toarray(), columns = cv.get_feature_names())

In [38]:
cv_flood_df.head()

,abandoned,abe,ability,able,absolutely,absorb,abundance,accept,access,accessible,...,wrought,yard,year,yes,yesterday,york,youre,zarif,zimbabwe,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,15,0,0,2,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,4,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,6,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
texts_flood = [cv_flood_df.columns[cv_flood_df.loc[index,:].nonzero()] for index in cv_flood_df.index]

/Users/evelyn/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  """Entry point for launching an IPython kernel.


In [41]:
texts_flood[:5]

[Index(['according', 'accounted', 'act', 'action', 'adapt', 'add', 'adequate',
        'administration', 'admit', 'africa',
        ...
        'wintertime', 'wiped', 'wise', 'won', 'world', 'worldwide', 'worst',
        'worth', 'year', 'york'],
       dtype='object', length=468),
 Index(['avoid', 'away', 'best', 'car', 'check', 'coming', 'danger', 'deadly',
        'deep', 'difference', 'doesn', 'dry', 'emergency', 'escape',
        'especially', 'event', 'explain', 'flash', 'float', 'flood', 'foot',
        'force', 'ground', 'guide', 'help', 'higher', 'home', 'importance',
        'important', 'know', 'land', 'learn', 'light', 'little', 'll', 'look',
        'make', 'pay', 'plus', 'prepared', 'raging', 'rain', 'reach', 'read',
        'remember', 'river', 'safely', 'safety', 'seeking', 'situation',
        'skilled', 'strike', 'surprised', 'swept', 'swimmer', 'threatening',
        'tip', 'trapped', 'tv', 'video', 'wait', 'want', 'warning', 'watch',
        'water', 'way'],
       

In [46]:
dictionary2 = corpora.Dictionary(texts_flood)

### Building Corpus 

In [47]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [48]:
corpus2 = [dictionary.doc2bow(text) for text in texts_flood]

### Running LDA with Num_topics = 2

In [23]:
ldamodel = models.ldamodel.LdaModel(corpus,                     # pass in our corpus
                                    id2word = dictionary,       # matches each word to its "number" or "spot" in the dictionary
                                    num_topics = 2,             # number of topics T to find
                                    passes = 5,                 # number of passes through corpus; similar to number of epochs
                                    minimum_probability = 0.01)

In [24]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

/Users/evelyn/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=             x    y  topics  cluster       Freq
topic                                          
0      0.09592  0.0       1        1  52.365719
1     -0.09592  0.0       2        1  47.634274, topic_info=     Category         Freq         Term        Total  loglift  logprob
42    Default  2391.000000    officials  2391.000000  30.0000  30.0000
236   Default  3225.000000     national  3225.000000  29.0000  29.0000
252   Default  1752.000000        winds  1752.000000  28.0000  28.0000
250   Default  2239.000000      weather  2239.000000  27.0000  27.0000
308   Default  1869.000000    residents  1869.000000  26.0000  26.0000
6     Default  2844.000000         area  2844.000000  25.0000  25.0000
325   Default  2358.000000        north  2358.000000  24.0000  24.0000
68    Default  2071.000000        storm  2071.000000  23.0000  23.0000
108   Default  2234.000000        areas  2234.000000  22.0000  22.0000
65    Default  2008.000000        south  2008.000000  21.0000  21.0000
300   Default  1866.000000        homes  1866.000000  20.0000  20.0000
231   Default  1978.000000        miles  1978.000000  19.0000  19.0000
54    Default  1645.000000       region  1645.000000  18.0000  18.0000
819   Default  3418.000000        state  3418.000000  17.0000  17.0000
334   Default  1497.000000        coast  1497.000000  16.0000  16.0000
378   Default  1282.000000     flooding  1282.000000  15.0000  15.0000
129   Default  1666.000000    emergency  1666.000000  14.0000  14.0000
156   Default  2617.000000         near  2617.000000  13.0000  13.0000
434   Default  1402.000000  authorities  1402.000000  12.0000  12.0000
52    Default  1413.000000         rain  1413.000000  11.0000  11.0000
464   Default  1846.000000       killed  1846.000000  10.0000  10.0000
66    Default  1448.000000     southern  1448.000000   9.0000   9.0000
1208  Default  1614.000000       caused  1614.000000   8.0000   8.0000
401   Default  1258.000000     northern  1258.000000   7.0000   7.0000
125   Default  1571.000000       county  1571.000000   6.0000   6.0000
175   Default  2164.000000     reported  2164.000000   5.0000   5.0000
106   Default  1503.000000       agency  1503.000000   4.0000   4.0000
321   Default  1528.000000         east  1528.000000   3.0000   3.0000
22    Default  1927.000000     expected  1927.000000   2.0000   2.0000
641   Default  2456.000000        water  2456.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
273    Topic2  3065.581299         like  4634.055664   0.3284  -5.9207
1182   Topic2   913.450378         real  1095.510864   0.5599  -7.1314
2177   Topic2   913.832886        doesn  1106.750854   0.5501  -7.1310
1932   Topic2  1598.394409         know  2198.437012   0.4229  -6.5719
2029   Topic2  1280.305786       really  1679.569214   0.4702  -6.7938
284    Topic2  1540.363159        right  2145.654297   0.4102  -6.6089
583    Topic2   947.591858      instead  1169.435303   0.5313  -7.0947
687    Topic2  1460.601929          don  2028.129883   0.4134  -6.6620
275    Topic2  1993.998413         make  3037.418457   0.3208  -6.3507
2161   Topic2  1576.635010         work  2272.010010   0.3762  -6.5856
274    Topic2  1222.828613         look  1640.563721   0.4477  -6.8397
223    Topic2  2667.305420         just  4620.696777   0.1921  -6.0598
1156   Topic2  1930.247192          way  3117.152588   0.2623  -6.3832
277    Topic2  2739.231689          new  5117.450195   0.1166  -6.0332
76     Topic2  2659.321533         time  4945.718750   0.1212  -6.0628
1819   Topic2  1260.212280      company  1738.707031   0.4198  -6.8096
1154   Topic2  1217.976440           ve  1668.204102   0.4271  -6.8437
780    Topic2  1659.333130        world  2665.768311   0.2675  -6.5345
685    Topic2  1071.507446    different  1400.729248   0.4737  -6.9718
727    Topic2  1351.859009         need  2129.918457   0.2870  -6.7394
1261   Topic2  1444.130127        going  2467.710205   0.2058  -6.6734
88